# Delta Lake Advanced

This notebook covers:
1. Implement incremental MERGE
2. Query historical versions
3. Optimize tables
4. Clean old files

In [0]:

# Use catalog & schema
spark.sql("USE CATALOG workspace")
spark.sql("USE SCHEMA ecommerce")


DataFrame[]

In [0]:
df = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data",
    header=True,
    inferSchema=True
)

df.printSchema()
df.show(5)


root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code| brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...|xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|janome|293.65|53

In [0]:

df.write.format("delta")     .mode("overwrite")     .saveAsTable("workspace.ecommerce.events_delta")


In [0]:

# Incremental updates DataFrame
updates_df = spark.createDataFrame(
    [
        (101, 2001, "2024-01-06 10:00:00", "purchase", 499.0),
        (102, 2003, "2024-01-06 11:00:00", "view", 0.0)
    ],
    ["user_id", "product_id", "event_time", "event_type", "price"]
)


In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(
    spark,
    "workspace.ecommerce.events_delta"
)

delta_table.alias("t").merge(
    updates_df.alias("s"),
    """
    t.user_id = s.user_id AND
    t.product_id = s.product_id AND
    t.event_time = s.event_time
    """
).whenMatchedUpdate(
    set={
        "event_type": "s.event_type",
        "price": "s.price"
    }
).whenNotMatchedInsert(
    values={
        "user_id": "s.user_id",
        "product_id": "s.product_id",
        "event_time": "s.event_time",
        "event_type": "s.event_type",
        "price": "s.price"
    }
).execute()


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:

# View history
spark.sql("DESCRIBE HISTORY workspace.ecommerce.events_delta").show()


+-------+-------------------+--------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|        userId|            userName|           operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+--------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      3|2026-01-13 11:34:13|78083193932656|vsskiranmai2005@g...|               MERGE|{predicate -> ["(...|NULL|{4044567713056020}|0113-112303-a71c5...|          2|WriteSerializable|        false|{numTargetRowsCop...|        NULL|Databricks-Run

In [0]:

v0 = spark.read.format("delta")     .option("versionAsOf", 0)     .table("workspace.ecommerce.events_delta")
v0.show()


+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:01|      view|  17302664|2053013553853497655|                NULL|   creed| 28.31|561587266|755422e7-9040-477...|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:01|      view|   1004775|2053013555631882655|electronics.s

In [0]:

# Optimize table
spark.sql("""
OPTIMIZE workspace.ecommerce.events_delta
ZORDER BY (event_type, user_id)
""")


DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [0]:

# Vacuum old files
spark.sql("VACUUM workspace.ecommerce.events_delta RETAIN 168 HOURS")


DataFrame[path: string]